In [1]:
import pandas as pd
import numpy as np
import pdfplumber as pf

In [2]:
scores = {
    '1' : 20, 
    '2' : 17, 
    '3' : 16, 
    '4' : 15, 
    '5' : 14,
    '6' : 13,
    '7' : 12,
    '8' : 11,
    '9' : 9,
    '10' : 7,
    '11' : 6,
    '12' : 5,
    '13' : 4,
    '14' : 3,
    '15' : 2,
    '16' : 1
}

In [3]:
psych_sheet = pf.open('di-men-championships-psych-sheet-updated-3-13-25-Copy1.pdf')

In [4]:
def get_line_locs(column, page_center, page):
    locs = []
    if (column=='left'):
        for item in page.filter(lambda obj : obj['x0'] < page_center).chars:
            locs.append(item['y0'])
    if (column=='right'):
        for item in page.filter(lambda obj : obj['x0'] > page_center).chars:
            locs.append(item['y0'])
    locs = list(set(locs))
    locs.sort(reverse=True)
    return locs

In [5]:
def line_to_text(column, line_loc, page_center, page):
    text = ""
    found_comma = False
    count = 0
    was_numeric = False
    if column=='right':
        for item in page.filter( lambda obj : ((obj['y0']==loc)&(obj['x0']>page_center)) ).chars:
            if (item['text'].isnumeric()):
                was_numeric = True
                text = text + str(item['text'])
            elif (was_numeric&(not item['text'].isnumeric())):
                was_numeric = False
                text = text + '|' + str(item['text'])
            else:
                text = text + str(item['text'])
            if (item['text']==','):
                found_comma = True
            if (found_comma):
                count = count+1
                if (count==3):
                    break
    if column=='left':
        for item in page.filter( lambda obj : ((obj['y0']==loc)&(obj['x0']<page_center)) ).chars:
            if (item['text'].isnumeric()):
                was_numeric = True
                text = text + str(item['text'])
            elif (was_numeric&(not item['text'].isnumeric())):
                was_numeric = False
                text = text + '|' + str(item['text'])
            else:
                text = text + str(item['text'])
            if (item['text']==','):
                found_comma = True
            if (found_comma):
                count = count+1
                if (count==3):
                    break
    return text

In [6]:
scorers = {}
doc = psych_sheet
page_center = 292.183557472
column_values = ['left', 'right']
for page in doc.pages:
    for column in column_values:
        locs = get_line_locs(column, page_center, page)
        for loc in locs:
            #parsing this column of the pdf into text lines. place and name will be delimited by a pipe
            text = line_to_text(column, loc, page_center, page)
            #print(text)

            #filling the scorers dictionary using the line of text
            line = text.split('|')
            place = line[0]
            if (place.isnumeric()):
                if (int(place)<17):
                    name = line[1]
                    if name in scorers.keys():
                        scorers[name] += scores[place]
                    else:
                        scorers[name] = scores[place]

In [7]:
psych_sheet.close()

In [12]:
scorers_df = pd.DataFrame(data={'name' : list(scorers.keys())[1:], 'score' : list(scorers.values())[1:]})

scorers_df.sort_values(by='score', ascending=False, inplace=True)
scorers_df.reset_index(drop=True, inplace=True)

scorers_df.to_csv('draft_board.csv')